In [ ]:
def pltm(history,save_dir):
    plt.figure(figsize=(16,5), dpi=85)
    ax1 = plt.subplot(121)#loss    
    plt.plot(history["loss"]["train"], label="train_loss")
    plt.plot(history["loss"]["val"], label="val_loss")
    plt.legend()  
    plt.xlabel('epoches')
    plt.title('Loss')
    
    ax2 = plt.subplot(122)#acc
    plt.plot(history["accuracy"]["train"], label="train_acc")
    plt.plot(history["accuracy"]["val"], label="val_acc")
    plt.legend()  
    plt.xlabel('epoches')
    plt.ylabel('%')
    plt.title('Acc')
#     plt.show
    plt.savefig(os.path.join(save_dir,'train-loss-acc-plt.png'))

def train( train_dataset, test_dataset,bestacc, model, max_epoches, learning_rate,batch_size, subdir):
    # train model
    history = {"accuracy": {"train": [],"val": [] },
               "loss": { "train": [], "val": [] },
               "lr":[learning_rate],
               'best_score':{'val':0,'test':0} }
    
    step_size=100
    gamma=0.8
    loss_fn =nn.CrossEntropyLoss()
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    optim = paddle.optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())
    lr_scheduler = paddle.optimizer.lr.StepDecay(learning_rate, step_size=step_size, gamma=gamma, last_epoch=- 1, verbose=False)

    save_dir = os.path.join('./checkpoint','{}'.format(subdir),'{}-{}-{}'.format(model.full_name(), max_epoches, learning_rate),)
    
    for epoch in range(max_epoches):
        print('**'*10,'Epoch {}/{}'.format(epoch+1,max_epoches),'**'*10)
        print('last_lr_rate:{} '.format(history['lr'][-1]))      
#         print('-' * 20)
        accuracy_recorder = {'train': [], 'val': []}
        loss_recorder = {'train': [], 'val': []}
        
        model.train()
        for tr_batch_id, (train_data, train_label) in enumerate(train_loader()) :   
            numb = int(train_data.shape[2]/2)
            predicts= model(train_data[:,:,:numb], train_data[:,:,numb:]) 
            loss = loss_fn(predicts, train_label)
            acc = paddle.static.accuracy(predicts, train_label)
            if (tr_batch_id)%10 == 0:
                print(".......: {}/{} \tLoss: {} \tAcc: {}".format(tr_batch_id+1, 
                            len(train_loader),"%.4f" % loss.numpy(),"%.3f" % acc.numpy()))            
                accuracy_recorder['train'].append(acc.item())
                loss_recorder['train'].append(loss.item())   
                        
            loss.backward()
            optim.step()
            optim.clear_grad()  
        lr_scheduler.step()  
        history["lr"].append(lr_scheduler.last_lr)
        print("     train: \t loss:[{}], \t Acc:[{}]".format("%.3f" % 
            np.mean(loss_recorder['train']),"%.4f" % np.mean(accuracy_recorder['train'])))

        model.eval()
        for ts_batch_id, (val_data, val_label) in enumerate(val_loader()):   
            numb = int(val_data.shape[2]/2)
            predicts= model(val_data[:,:,:numb], val_data[:,:,numb:]) 
            loss = loss_fn(predicts, val_label)
            acc = paddle.static.accuracy(predicts, val_label)
            
            accuracy_recorder['val'].append(acc.item())
            loss_recorder['val'].append(loss.item())
        print("     val:   \t loss:[{}], \t Acc:[{}]".format("%.3f" % 
              np.mean(loss_recorder['val']),"%.4f" % np.mean(accuracy_recorder['val'])))
        
        
        #BEST MODEL
        score = np.mean(accuracy_recorder['val'])

        if score > history['best_score']['val']:
            history['best_score']['val'] = score           
            bestacc.append(score)

            paddle.save(model.state_dict(), os.path.join(save_dir,'best_model.pdparams'.format(history['best_score'])))

            best_model = model.state_dict()
            print('<<<Save>>><<<Save>>><<<Save>>>best_Acc:{}'.format("%.2f" %score))
        # record loss and accuracy
        history["loss"]["train"].append(np.mean(loss_recorder["train"]))
        history["loss"]["val"].append(np.mean(loss_recorder["val"]))
        history["accuracy"]["train"].append(np.mean(accuracy_recorder["train"]))
        history["accuracy"]["val"].append(np.mean(accuracy_recorder["val"]))
    model.set_state_dict(best_model)

    with open(os.path.join(save_dir,'history-{}.pkl'.format(model.full_name())), 'wb+') as f:
            pickle.dump(history, f)

    print('best score:{}'.format(history['best_score']['val']))
    pltm(history,save_dir)